# 1

%md
You are given a dataset containing sales data for different stores across various months. Each row contains the store name, the month, and the sales amount. Your task is to calculate the cumulative sales for each store, considering the monthly sales, using PySpark.

You should also:
Filter out stores with sales lower than 1000 in any month.
Calculate the total sales for each store over all months.
Sort the results by the total sales in descending order.

data = [ ("Store A", "2024-01", 800), ("Store A", "2024-02", 1200), ("Store A", "2024-03", 900), ("Store B", "2024-01", 1500), ("Store B", "2024-02", 1600), ("Store B", "2024-03", 1400), ("Store C", "2024-01", 700), ("Store C", "2024-02", 1000), ("Store C", "2024-03", 800) ] 

Create DataFrame df = spark.createDataFrame(data, ["Store", "Month", "Sales"]) 

# 2

You are given a dataset containing sales data for different stores across various months. Each row contains the store name, the month, and the sales amount. Your task is to calculate the cumulative sales for each store, considering the monthly sales, using PySpark.

You should also:
Filter out stores with sales lower than 1000 in any month.
Calculate the total sales for each store over all months.
Sort the results by the total sales in descending order.

𝐬𝐜𝐡𝐞𝐦𝐚 𝐚𝐧𝐝 𝐝𝐚𝐭𝐚𝐬𝐞𝐭
data = [ ("Store A", "2024-01", 800), ("Store A", "2024-02", 1200), ("Store A", "2024-03", 900), ("Store B", "2024-01", 1500), ("Store B", "2024-02", 1600), ("Store B", "2024-03", 1400), ("Store C", "2024-01", 700), ("Store C", "2024-02", 1000), ("Store C", "2024-03", 800) ] 

Create DataFrame df = spark.createDataFrame(data, ["Store", "Month", "Sales"]) 

# 3

You are given two datasets — one containing employee details such as name, department, salary, and joining date, and another containing their job titles. Your task is to identify the employees who have the highest and lowest salary across the company. The final output should display their name, job title, salary, and a new column called salary_type that indicates whether the salary is the "Highest Salary" or the "Lowest Salary".

worker:
|worker_id|first_name|last_name|salary|joining_date| department|
| 1| John| Doe| 5000| 2023–01–01|Engineering|
| 2| Jane| Smith| 6000| 2023–01–15| Marketing|
| 3| Alice| Johnson| 4500| 2023–02–05|Engineering|


title:
|worker_ref_id|worker_title|affected_from|
| 1| Engineer| 2022–01–01|
| 2| Manager| 2022–01–01|
| 3| Engineer| 2022–01–01|

In [0]:
from pyspark.sql.functions import col, when
from pyspark.sql.window import Window

# Create DataFrames
worker_data = [
    (1, "John", "Doe", 5000, "2023-01-01", "Engineering"),
    (2, "Jane", "Smith", 6000, "2023-01-15", "Marketing"),
    (3, "Alice", "Johnson", 4500, "2023-02-05", "Engineering")
]
title_data = [
    (1, "Engineer", "2022-01-01"),
    (2, "Manager", "2022-01-01"),
    (3, "Engineer", "2022-01-01")
]

df_worker = spark.createDataFrame(worker_data, ["worker_id", "first_name", "last_name", "salary", "joining_date", "department"])
df_title = spark.createDataFrame(title_data, ["worker_ref_id", "worker_title", "affected_from"])

# Use window functions to rank by salary
from pyspark.sql.functions import dense_rank
window_spec = Window.orderBy(col("salary").desc())

df_ranked = df_worker.withColumn("rank_high", dense_rank().over(window_spec)) \
                     .withColumn("rank_low", dense_rank().over(Window.orderBy(col("salary").asc())))

# Filter highest and lowest salaries
df_filtered = df_ranked.filter((col("rank_high") == 1) | (col("rank_low") == 1))



In [0]:
df_filtered.display()

worker_id,first_name,last_name,salary,joining_date,department,rank_high,rank_low
3,Alice,Johnson,4500,2023-02-05,Engineering,3,1
2,Jane,Smith,6000,2023-01-15,Marketing,1,3


In [0]:
df_filtered=df_filtered.withColumn("salary_type",when(col("rank_high")==1,"Highest Salary").when(col("rank_low")==1,"Lowest_Salary"))


In [0]:
df_filtered.display()

worker_id,first_name,last_name,salary,joining_date,department,rank_high,rank_low,salary_type
3,Alice,Johnson,4500,2023-02-05,Engineering,3,1,Lowest_Salary
2,Jane,Smith,6000,2023-01-15,Marketing,1,3,Highest Salary


In [0]:
df_final=df_filtered.join(df_title,df_filtered.worker_id==df_title.worker_ref_id,"inner")\
    .select("worker_id", "first_name", "last_name", "salary", "worker_title", "salary_type")\
    .display()

worker_id,first_name,last_name,salary,worker_title,salary_type
3,Alice,Johnson,4500,Engineer,Lowest_Salary
2,Jane,Smith,6000,Manager,Highest Salary


# 4
Pivot the Column into Rows
𝐬𝐜𝐡𝐞𝐦𝐚 𝐚𝐧𝐝 𝐝𝐚𝐭𝐚𝐬𝐞𝐭
Given Input -

StudentID, StudentName , AScore, BScore,CScore
123, A, 30, 31, 32
124, B, 40, 41, 42
Get the output in below format -

StudentID, StudentName , Subject , Score
123, A, AScore, 30
123, A, BScore, 31
123, A, CScore, 32
124, B, AScore, 40
124, B, BScore, 41
124, B, CScore, 42

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr

# Create Spark session
spark = SparkSession.builder.appName("PivotColumnsToRows").getOrCreate()

# Input Data
data = [
    (123, "A", 30, 31, 32),
    (124, "B", 40, 41, 42)
]
columns = ["StudentID", "StudentName", "AScore", "BScore", "CScore"]

# Create DataFrame
df = spark.createDataFrame(data, columns)

In [0]:
df.display()

StudentID,StudentName,AScore,BScore,CScore
123,A,30,31,32
124,B,40,41,42


In [0]:
df_unpivoted = df.selectExpr(
    "StudentID", "StudentName",
    "stack(3, 'AScore', AScore, 'BScore', BScore, 'CScore', CScore) as (Subject, Score)"
)

In [0]:
df_unpivoted.display()

StudentID,StudentName,Subject,Score
123,A,AScore,30
123,A,BScore,31
123,A,CScore,32
124,B,AScore,40
124,B,BScore,41
124,B,CScore,42


# 5
Handle Missing Data Efficiently
You have a DataFrame containing sales data. Write a PySpark script to:
Replace missing values in the price column with the mean price.
Drop rows where the product column is null.
Fill missing values in the quantity column with 1.

data = [ (1, "Laptop", 1000, 5), (2, "Mouse", None, None), (3, "Keyboard", 50, 2), (4, "Monitor", 200, None), (5, None, 500, None), ] 

Define schema and create DataFrame 
columns = ["product_id", "product", "price", "quantity"]

In [0]:
data = [
    (1, "Laptop", 1000, 5),
    (2, "Mouse", None, None),
    (3, "Keyboard", 50, 2),
    (4, "Monitor", 200, None),
    (5, None, 500, None),
]
columns = ["product_id", "product", "price", "quantity"]

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg

# Create Spark session
spark = SparkSession.builder.getOrCreate()

# Create DataFrame
df = spark.createDataFrame(data, columns)

You are given a dataset containing sales information with some missing values. Your task is to handle these nulls efficiently using PySpark:

Replace missing values in the price column with the average price.

Drop any row where the product column is null.

Fill missing values in the quantity column with 1.



In [0]:
df.display()

product_id,product,price,quantity
1,Laptop,1000,5
2,Mouse,null,null
3,Keyboard,50,2
4,Monitor,200,null
5,null,500,null


In [0]:
df=df.dropna(subset=["product"])
df.display()

product_id,product,price,quantity
1,Laptop,1000,5
2,Mouse,null,null
3,Keyboard,50,2
4,Monitor,200,null


In [0]:
df_avg=df.select(avg("price")).first()[0]
df=df.fillna({"price":df_avg})

In [0]:
df.display()


product_id,product,price,quantity
1,Laptop,1000,5
2,Mouse,416,null
3,Keyboard,50,2
4,Monitor,200,null


In [0]:
df=df.fillna({"quantity":1})
df.display()

product_id,product,price,quantity
1,Laptop,1000,5
2,Mouse,416,1
3,Keyboard,50,2
4,Monitor,200,1


# 6

You are provided with a DataFrame containing customer purchase data. Your task is to:
Identify and remove duplicate rows based on the customer_id column.
If duplicates exist for a customer_id, retain the row with the most recent purchase_date.



data = [ (1, "Laptop", "2024-08-01"), (1, "Mouse", "2024-08-05"), (2, "Keyboard", "2024-08-02"), (2, "Monitor", "2024-08-03") ] 

columns = ["customer_id", "product", "purchase_date"]

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, row_number
from pyspark.sql.window import Window

# Create Spark session
spark = SparkSession.builder.getOrCreate()

# Sample data
data = [
    (1, "Laptop", "2024-08-01"),
    (1, "Mouse", "2024-08-05"),
    (2, "Keyboard", "2024-08-02"),
    (2, "Monitor", "2024-08-03")
]

columns = ["customer_id", "product", "purchase_date"]

# Create DataFrame
df_6 = spark.createDataFrame(data, columns)


In [0]:
df_6.display()

customer_id,product,purchase_date
1,Laptop,2024-08-01
1,Mouse,2024-08-05
2,Keyboard,2024-08-02
2,Monitor,2024-08-03


In [0]:
df_6=df_6.withColumn("purchase_date",col("purchase_date").cast("date"))\
    .withColumn("rank",row_number().over(Window.partitionBy("customer_id").orderBy(col("purchase_date").desc())))\
    .filter(col("rank")==1)\
    .display()

customer_id,product,purchase_date,rank
1,Mouse,2024-08-05,1
2,Monitor,2024-08-03,1


# 7.
You are given product and price data:

Add a new column price_category using the rules:

"Low" if price < 100

"Medium" if 100 ≤ price < 500

"High" if price ≥ _500_


data = [("Laptop", 800), ("Mouse", 25), ("Keyboard", 150), ("Monitor", 300)] 

columns = ["product", "price"] [](url)

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import when, col

# Start Spark session
spark = SparkSession.builder.getOrCreate()

# Sample data
data = [("Laptop", 800), ("Mouse", 25), ("Keyboard", 150), ("Monitor", 300)]
columns = ["product", "price"]

# Create DataFrame
df_7 = spark.createDataFrame(data, columns)

---------------------------------------------------------------------------
_InactiveRpcError                         Traceback (most recent call last)
File /databricks/python/lib/python3.11/site-packages/pyspark/sql/connect/client/core.py:1853, in SparkConnectClient.config(self, operation)
   1852 with attempt:
-> 1853     resp = self._stub.Config(req, metadata=self.metadata())
   1854     self._verify_response_integrity(resp)

File /databricks/python/lib/python3.11/site-packages/grpc/_interceptor.py:277, in _UnaryUnaryMultiCallable.__call__(self, request, timeout, metadata, credentials, wait_for_ready, compression)
    268 def __call__(
    269     self,
    270     request: Any,
   (...)
    275     compression: Optional[grpc.Compression] = None,
    276 ) -> Any:
--> 277     response, ignored_call = self._with_call(
    278         request,
    279         timeout=timeout,
    280         metadata=metadata,
    281         credentials=credentials,
    282         wait_for_ready=wai

In [0]:
df_7.display()

product,price
Laptop,800
Mouse,25
Keyboard,150
Monitor,300


In [0]:
df_7=df_7.withColumn("Price_catagory",
                     when(col("price")>100,"low")
                     .when((col("price")>100) & (col("price")<500),'Mddium')
                     .otherwise("High")
                     
                     )

com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$5(SequenceExecutionState.scala:132)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:132)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:129)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:129)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:715)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:435)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:435)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.can

In [0]:
df_7.display()

com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:434)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.cancelExecution(ExecutionContextManagerV1.scala:466)
	at com.databricks.spark.chauffeur.ChauffeurState.$anonfun$process$1(ChauffeurState.scala:757)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperation$1(UsageLogging.scala:510)
	at com.databricks.logging.UsageLogging.executeThunkAndCaptureResultTags$1(UsageLogging.scala:616)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperationWithResultTags$4(UsageLogging.scala:643)
	at com.databricks.logging.AttributionContextTracing.$anonfun$withAttributionContext$1(AttributionContextTracing.scala:49)
	at com.databricks.logging.AttributionContext$.$anonfun$withValue$1(AttributionContext.scala:293)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.AttributionContext$.withValue(Attr

# 8 
Given employee data with columns:
employee_id, department, salary, join_date

You need to:

Calculate cumulative salary within each department, ordered by join_date.

Assign dense rank within each department based on salary in descending order.

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum as _sum
from pyspark.sql.window import Window
from pyspark.sql.functions import dense_rank

# Start Spark session
spark = SparkSession.builder.getOrCreate()

# Sample data
data = [
    (1, "HR", 50000, "2022-01-01"),
    (2, "IT", 70000, "2021-06-15"),
    (3, "HR", 60000, "2023-03-10"),
    (4, "IT", 80000, "2020-12-01")
]
columns = ["employee_id", "department", "salary", "join_date"]

# Create DataFrame
df_8 = spark.createDataFrame(data, columns)

---------------------------------------------------------------------------
_InactiveRpcError                         Traceback (most recent call last)
File /databricks/python/lib/python3.11/site-packages/pyspark/sql/connect/client/core.py:1853, in SparkConnectClient.config(self, operation)
   1852 with attempt:
-> 1853     resp = self._stub.Config(req, metadata=self.metadata())
   1854     self._verify_response_integrity(resp)

File /databricks/python/lib/python3.11/site-packages/grpc/_interceptor.py:277, in _UnaryUnaryMultiCallable.__call__(self, request, timeout, metadata, credentials, wait_for_ready, compression)
    268 def __call__(
    269     self,
    270     request: Any,
   (...)
    275     compression: Optional[grpc.Compression] = None,
    276 ) -> Any:
--> 277     response, ignored_call = self._with_call(
    278         request,
    279         timeout=timeout,
    280         metadata=metadata,
    281         credentials=credentials,
    282         wait_for_ready=wai